In [ ]:
import os
import argparse
# internal imports
import tensorflow as tf

from magenta.music import midi_io
from magenta.music import musicxml_reader
from magenta.music import note_sequence_io

In [ ]:
def convert_midi(full_file_path):
  """Converts a midi file to a sequence proto."""
  try:
    sequence = midi_io.midi_to_sequence_proto(
        tf.gfile.FastGFile(full_file_path, 'rb').read())
  except midi_io.MIDIConversionError as e:
    tf.logging.warning(
        'Could not parse MIDI file %s. It will be skipped. Error was: %s',
        full_file_path, e)
    return None
  sequence.filename = full_file_path
  sequence.id = note_sequence_io.generate_note_sequence_id(
      sequence.filename, sequence.collection_name, 'midi')
  tf.logging.info('Converted MIDI file %s.', full_file_path)
#   print(sequence)
  return sequence
print('finish load')

In [ ]:
two_hand_path = '/Users/mac/Desktop/TwoHands.mid'
two_hand = convert_midi(two_hand_path)

In [ ]:
#Data Structure of Note:https://github.com/tensorflow/magenta/blob/master/magenta/protobuf/music.proto
print(len(two_hand.notes))
for i in range(10):
    print(two_hand.notes[i])
# print(res.tempos)
# print(res.PitchName.F_FLAT_FLAT)

In [ ]:
import midi
# Instantiate a MIDI Pattern (contains a list of tracks)
pattern = midi.Pattern()
# Instantiate a MIDI Track (contains a list of MIDI events)
track = midi.Track()
# Append the track to the pattern
pattern.append(track)
# Instantiate a MIDI note on event, append it to the track
on = midi.NoteOnEvent(tick=0, velocity=20, pitch=midi.G_3)
track.append(on)
# Instantiate a MIDI note off event, append it to the track
off = midi.NoteOffEvent(tick=100, pitch=midi.G_3)
track.append(off)
# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=1)
track.append(eot)
# Print out the pattern
print(pattern)
# Save the pattern to disk
midi.write_midifile("example.mid", pattern)

In [ ]:
import numpy as np
import tensorflow as tf
import keras
import os
import pickle
from my_to_midi import *
from sequence_example_lib import *
import copy
import pickle as pkl

In [ ]:
def to_events(x):
    return np.argmax(x, axis=1)
def to_real_length(x):
    while(x[len(x)-1]==0):
        x.pop()
    # delete the last note along with padded zeros
    x.pop()

In [ ]:
def sequence_example_to_real_inputs(sequence_example_path):
    # train
    sequence_example_file = sequence_example_path
    sequence_example_file_paths = tf.gfile.Glob(
        os.path.expanduser(sequence_example_file))
    start_time = time.time()
    inputs, labels, lengths = get_numpy_from_tf_sequence_example( input_size=259,
                                        sequence_example_file_paths = sequence_example_file_paths,
                                        shuffle = False)
    print('Time:',time.time() - start_time)
    print('inputs shape',inputs.shape)
    print('inputs type',type(inputs))
    input_events = []
    for i in inputs:
        input_events.append(to_events(i))
    real_inputs = []
    for i in input_events:
        d = []
        d = list(i)
        to_real_length(d)
        real_inputs.append(d)
    return real_inputs

In [4]:
def get_start(events):
    for i, event in enumerate(events):
        if event==2:
            return events[i:len(events)]
def get_melody(events):
    ret = []
    for i, event in enumerate(events):
        if event==2:
            pass
            #ret.append(event)
        else:
            if i>0 and events[i-1]==2:
                ret.append([event])
    return ret
def get_accomp(events):
    ret = []
    for i, event in enumerate(events):
        if event==2:
            pass
            #ret.append(event)
        else:
            if i>0 and events[i-1]==2:
                for j in range(i+1, len(events)):
                    if events[j]==2 or j==len(events)-1:
                        ret.append(events[i+1:j+1])
    return ret

In [5]:
def sequence_example_to_events_file(sequence_example_path, output_dir, name):
    real_inputs = sequence_example_to_real_inputs(sequence_example_path)
    
    melodys = []
    accomps = []
    
    for i, real_input in enumerate(real_inputs):
        events = get_start(real_input)
        melody = get_melody(events)
        accomp = get_accomp(events)
        melodys.append(melody)
        accomps.append(accomp)
        if i==0:
            print(events)
            print(melody)
            print(accomp)
    
    melody_path = os.path.join(output_dir, name)+'_melody.pkl'
    with open(melody_path,'wb') as mf:   #pickle只能以二进制格式存储数据到文件
        mf.write(pickle.dumps(melodys))   #dumps序列化源数据后写入文件
        mf.close()

    accomp_path = os.path.join(output_dir, name)+'_accomp.pkl'
    with open(accomp_path,'wb') as af:   #pickle只能以二进制格式存储数据到文件
        af.write(pickle.dumps(accomps))   #dumps序列化源数据后写入文件
        af.close()

In [6]:
def to_melody_and_accompaniment(sequence_example_dir, output_dir, name):
    train_path = os.path.join(sequence_example_dir, "training_poly_tracks.tfrecord")
    sequence_example_to_events_file(train_path, output_dir, name+"_train")
    eval_path = os.path.join(sequence_example_dir, "eval_poly_tracks.tfrecord")
    sequence_example_to_events_file(eval_path, output_dir, name+"_eval")

In [7]:
magenta_datasets_dirs = [
    '/Users/mac/Desktop/Bach/'
    ]
magenta_datasets_names = [
    'Bach'
    ]
for dataset_dir, dataset_name in zip(magenta_datasets_dirs, magenta_datasets_names):
    print("Converting:", dataset_dir, dataset_name)
    to_melody_and_accompaniment(dataset_dir, "/Users/mac/Desktop/makedataset", dataset_name)

Converting: /Users/mac/Desktop/Bach/ Bach
INFO:tensorflow:Counting records in /Users/mac/Desktop/Bach/training_poly_tracks.tfrecord.
INFO:tensorflow:Total records: 1998
INFO:tensorflow:[<tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:0' shape=(?, 259) dtype=float32>, <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>, <tf.Tensor 'strided_slice:0' shape=() dtype=int32>]
inputs (1998, 2395, 259) <class 'numpy.ndarray'> [ 0  2  2 71  2 70  2 71  2 73]
labels (1998, 2395) <class 'numpy.ndarray'> [ 2  2 71  2 70  2 71  2 73  2]
Time: 10.879290103912354
inputs shape (1998, 2395, 259)
inputs type <class 'numpy.ndarray'>


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 